In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
import unicodedata

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Input, Conv2D, Concatenate, MaxPool2D, Reshape, Flatten, Dropout
from keras.layers import ReLU
from keras.layers import BatchNormalization, LSTM, Bidirectional

from keras.callbacks import CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras.layers import Add

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.manifold import TSNE
from keras import backend as K

In [4]:
# !python3 -m pip install gensim

In [5]:
from gensim.models import Word2Vec

In [6]:
%matplotlib inline

In [7]:
# !python3 -m pip install -U nltk
# !python3 -m nltk.downloader punkt

In [8]:
import nltk

In [9]:
from nltk.corpus import stopwords

In [10]:
path = 'recipes_with_calories.csv'

In [11]:
df = pd.read_csv(path, sep="|", index_col=0)

In [12]:
all_columns = list(df.columns)
len(all_columns)

680

In [13]:
df.dropna(axis=0).shape, df.shape

((15864, 680), (20052, 680))

In [14]:
for column in df.columns:
    if df[column].isnull().any():
        print(column)
#         df[column] = df[column].fillna(df[column].mean())

protein
fat
sodium


In [16]:
def get_small_dict(df, keep_columns, index):
    my_dict = df.iloc[index][keep_columns].to_dict()
    my_dict['tags'] = [
        key for key, value in df.iloc[index].to_dict().items() if value != 0 and key not in keep_columns]
    my_dict['title'] = my_dict['title'].strip()
    my_dict['tags'] = [
        str(unicodedata.normalize('NFKD', t).encode('ascii','ignore'), encoding='ascii') for t in my_dict['tags']
    ]
    return my_dict

In [17]:
keep_columns = ['title', 'rating', 'calories', 'protein', 'fat','sodium']

In [18]:
def remove_outliers_with_tukey(df):
    print('There were {} samples before outlier removal.'.format(len(df)))

    # Remove calories outliers by the Tukey rule
    Q3 = np.percentile(df[regr_column], 75)
    Q1 = np.percentile(df[regr_column], 25)
    IQR = Q3 - Q1

    outliers = df[regr_column].loc[
        (df[regr_column] < (Q1 - 1.5 * IQR)) | 
        (df[regr_column] > (Q3 + 1.5 * IQR))
    ].index
    
    df = df.drop(outliers)
    
    print('There are now {} samples.'.format(len(cut_df)))

    return df

In [19]:
# pick regression column
regr_column = 'protein'
cut_df = df[df[regr_column] == df[regr_column]]

cut_df = remove_outliers_with_tukey(df=df)

# transform df in list of dictionaries for processing
regr_data = []
for i in range(len(cut_df)):
    regr_data.append(get_small_dict(cut_df, keep_columns, i))

There were 15890 samples before outlier removal.
There are now 15890 samples.


In [21]:
def remove_punctuation_signs(regr_data):
    for d in regr_data:
        if 'tags' not in d:
            raise ValueError('Expected to have <tags> key in each of the regression input data dictionary!')
    corpus = ['|'.join(
        d['tags'] + 
        [w.lower() for w in nltk.word_tokenize(d['title']) if
             w not in '''!()-[]{};:'"\,<>./?@#$%^&*_~''']
    ) for d in regr_data]
    return corpus

In [22]:
# make sure the data doesn't have any punctuation signs
corpus = remove_punctuation_signs(regr_data)

In [23]:
# pick tokenization method
def tokenize(word_list):
    return word_list.split('|')

In [24]:
def get_y_labels(regr_column, regr_data):
    return [d[regr_column] for d in regr_data]

In [25]:
y_label = get_y_labels(regr_column, regr_data)

In [26]:
if len(corpus) != len(y_label):
    print('[ERROR] There is a mismatch between the size of the input data ' +
          ' ({} elements) and the size of labels ({} elements)!'.format(len(corpus), len(y_label)))

## Try a vanilla regression (CountVectorizer + LinearRegression from sklearn)

In [27]:
# fit vectorizer
vectorizer = CountVectorizer(tokenizer=tokenize)
X = vectorizer.fit_transform(corpus)

# train, test data
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), y_label, test_size=0.2, random_state=42)

# sklearn LinearRegression model
regr_sklearn = LinearRegression()
regr_sklearn.fit(X_train, y_train)

# predict data
y_pred = regr_sklearn.predict(X_test)

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred)))

# show first k differences
first_k = 10
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  1.4353839059169882e+22
18.89750363295439 12.0
11.564207810810837 8.0
8.126576203755173 13.0
11.729228753498825 4.0
-49846010989.26628 2.0
7.23050381605986 3.0
2.9005344099319306 3.0
0.5286853498733368 4.0
18.47845723097685 20.0
38.56466141947109 35.0


## Try Word2Vec embeddings + Keras NN

### Word2Vec embedding

In [28]:
vocab = set()
for word_list in corpus:
    vocab |= set(tokenize(word_list))

print("Choose embedding size between {} and {}".format(np.log2(len(vocab)), np.power(len(vocab), 1/4) * 10))

w2v_corpus = []
for word_list in corpus:
    w2v_corpus.append(tokenize(word_list))

Choose embedding size between 12.523316912312747 and 87.59382856164761


In [29]:
EMBEDDING_SIZE = 50

In [30]:
w2v_model = Word2Vec(w2v_corpus, size=EMBEDDING_SIZE, window=5, min_count=1, workers=-1)

print("Embedding size is {}".format(w2v_model.wv.vector_size))

Embedding size is 50


In [31]:
sequence_lengths = [len(w) for w in w2v_corpus]

print("Sequence length should be {} ".format(pd.Series(sequence_lengths).describe()['mean']))

Sequence length should be 17.03228863467136 


In [32]:
FIXED_LENGTH = 17

In [33]:
def generate_embeddings_matrix(sample, model, fixed_length=17):
    embedding_size = model.wv.vector_size
    embeddings = []
    for sen in sample:
        for word in sen:
            if word not in model.wv.vocab:
                continue
            embeddings.append(model.wv[word])
            if len(embeddings) >= fixed_length:
                break

    embeddings = np.array(embeddings)

    while len(embeddings) < fixed_length:
        padding_length = fixed_length - len(embeddings)
        embeddings = np.concatenate((embeddings, embeddings[:padding_length]))
    if len(embeddings) > fixed_length:
        embeddings = embeddings[:fixed_length]

    return embeddings

In [34]:
def generate_input_matrix(sample, fixed_length=17):
    
    inputs = np.array(sample)

    while len(inputs) < fixed_length:
        padding_length = fixed_length - len(inputs)
        inputs = np.concatenate((inputs, inputs[:padding_length]))
    if len(inputs) > fixed_length:
        inputs = inputs[:fixed_length]

    return inputs

In [35]:
# Generate list of input data (embedding matrix) and make sure every input has FIXED_LENGTH elements
X = []
for sample in w2v_corpus:
    embedded_sample = generate_embeddings_matrix(sample, w2v_model, FIXED_LENGTH)
    if embedded_sample.shape != (FIXED_LENGTH, EMBEDDING_SIZE):
        print(embedded_sample.shape)
    X.append(embedded_sample)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.2, random_state=42)

### Keras NN

In [37]:
FIXED_LENGTH = 17
EMBEDDING_SIZE = 50

BATCH_SIZE = 16
EPOCHS=200

In [38]:
def baseline_model():
    # Create model
    input_layer = Input(shape=(FIXED_LENGTH, EMBEDDING_SIZE), name='input')
    flat = Flatten()(input_layer)
    z = Dense(32, activation='relu', name='fc1', kernel_initializer='normal')(flat)
    output = Dense(1, kernel_initializer='normal')(z)
    model = Model(inputs=[input_layer], outputs=[output])
    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [39]:
model = baseline_model()

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.2, random_state=42)

In [41]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [42]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/200
11818/11818 [==============================] - 1s 124us/step - loss: 284.0700
Epoch 2/200
11818/11818 [==============================] - 2s 134us/step - loss: 220.7680
Epoch 3/200
11818/11818 [==============================] - 2s 134us/step - loss: 215.2818
Epoch 4/200
11818/11818 [==============================] - 2s 144us/step - loss: 212.3870
Epoch 5/200
11818/11818 [==============================] - 1s 109us/step - loss: 210.5811
Epoch 6/200
11818/11818 [==============================] - 1s 111us/step - loss: 209.2927
Epoch 7/200
11818/11818 [==============================] - 1s 110us/step - loss: 208.3562
Epoch 8/200
11818/11818 [==============================] - 1s 111us/step - loss: 207.5347
Epoch 9/200
11818/11818 [==============================] - 2s 129us/step - loss: 206.9936
Epoch 10/200
11818/11818 [==============================] - 2s 128us/step - loss: 206.3022
Epoch 11/200
11818/11818 [==============================] - 1s 124us/step - loss: 205.9096
Epoch 12

11818/11818 [==============================] - 1s 94us/step - loss: 202.9089
Epoch 92/200
11818/11818 [==============================] - 1s 92us/step - loss: 202.9508
Epoch 93/200
11818/11818 [==============================] - 1s 97us/step - loss: 202.8762
Epoch 94/200
11818/11818 [==============================] - 1s 99us/step - loss: 202.8900
Epoch 95/200
11818/11818 [==============================] - 1s 90us/step - loss: 202.9161
Epoch 96/200
11818/11818 [==============================] - 1s 92us/step - loss: 202.8633
Epoch 97/200
11818/11818 [==============================] - 1s 93us/step - loss: 202.8758
Epoch 98/200
11818/11818 [==============================] - 1s 93us/step - loss: 202.8294
Epoch 99/200
11818/11818 [==============================] - 1s 91us/step - loss: 202.7931
Epoch 100/200
11818/11818 [==============================] - 1s 92us/step - loss: 202.8453
Epoch 101/200
11818/11818 [==============================] - 1s 96us/step - loss: 202.7533
Epoch 102/200
11818/1

11818/11818 [==============================] - 1s 97us/step - loss: 198.8725
Epoch 182/200
11818/11818 [==============================] - 1s 95us/step - loss: 198.9415
Epoch 183/200
11818/11818 [==============================] - 1s 96us/step - loss: 198.8238
Epoch 184/200
11818/11818 [==============================] - 1s 92us/step - loss: 198.7341
Epoch 185/200
11818/11818 [==============================] - 1s 98us/step - loss: 198.6417
Epoch 186/200
11818/11818 [==============================] - 1s 98us/step - loss: 198.6823
Epoch 187/200
11818/11818 [==============================] - 1s 96us/step - loss: 198.5848
Epoch 188/200
11818/11818 [==============================] - 1s 92us/step - loss: 198.5182
Epoch 189/200
11818/11818 [==============================] - 1s 91us/step - loss: 198.3366
Epoch 190/200
11818/11818 [==============================] - 1s 93us/step - loss: 198.3881
Epoch 191/200
11818/11818 [==============================] - 1s 97us/step - loss: 198.3181
Epoch 192/200

In [51]:
y_pred = model.predict(X_test)

y_pred = y_pred.flatten()

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred)))
print("Std: ", np.std(y_test))
print("Mean: ", np.mean(y_test))

# show first k differences
first_k = 10
print("Showing the first {} differences between predicted and real.".format(first_k))
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  14.606599555154107
Std:  15.518228806405345
Mean:  13.820304568527918
Showing the first 10 differences between predicted and real.
14.127321 12.0
13.207834 8.0
16.411182 13.0
12.423544 4.0
9.925699 2.0
10.550068 3.0
4.9842587 3.0
10.045847 4.0
15.637499 20.0
20.618483 35.0


### Save model

In [52]:
model.save('models/predict_protein_model')

### Get the dataframe that has NaN values for our regression column (calories)

In [53]:
predict_df = df[df[regr_column] != df[regr_column]]

### Get the input data we need to predict the needed values

In [54]:
predict_data = []
for i in range(len(predict_df)):
    predict_data.append(get_small_dict(predict_df, keep_columns, i))
corpus = ['|'.join(
    d['tags'] + 
    [w.lower() for w in nltk.word_tokenize(d['title']) if w not in '''!()-[]{};:'"\,<>./?@#$%^&*_~''']
) for d in predict_data]

### Split input data using the tokenizer

In [55]:
predict_corpus = []
for word_list in corpus:
    predict_corpus.append(tokenize(word_list))

### Make sure input data has a fixed shape

In [63]:
X = []
for sample in predict_corpus:
    sample = [w for w in sample if w in w2v_model.wv]
    input_sample = generate_embeddings_matrix(sample, w2v_model, FIXED_LENGTH)
    if input_sample.shape != (FIXED_LENGTH, EMBEDDING_SIZE):
        print(input_sample.shape)
    X.append(input_sample)

### Predict missing data 

In [70]:
y_pred = model.predict(np.array(X))

y_pred = y_pred.flatten()

### Fill the dataframe with predicted values

In [72]:
predict_df[regr_column] = y_pred

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Update initial dataframe with missing values

In [75]:
update_regr_column = [np.nan for _ in range(len(df))]
for idx, value in zip(predict_df.index, y_pred):
    update_regr_column[idx] = value

In [76]:
df.update({regr_column: update_regr_column})

### Save updated dataframe

In [78]:
df.to_csv('recipes_with_calories_and_protein.csv', sep='|')